## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [11]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSBSSSSB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
103
+-------------------+------------------+------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|            coeff_1|           coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|          res_mean|    station|total_var|undefs|              vector|  year|
+-------------------+------------------+------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|  36.88280420965144|0.9302361398007852|0.1591972219710314|    152.7|BSBSSSSB| 32.6089| -85.0756|       SNWD|0.9272904811360495|0.9272862359830134|0.9272845486974798|0.9952366993333596|USC00096148|

In [12]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [42]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1 FROM weather


station    year    coeff_1
0  USC00096148  2002.0  36.882804
1  USC00097087  2010.0  -0.959282
2  USC00016129  1973.0  -1.925512
3  USC00093544  1955.0   3.818167
4  USC00093544  1967.0  -1.139350

In [43]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')
year_station_table.head(10)

station  US1GATY0002  USC00010430  USC00011725  USC00012730  USC00014080  \
year                                                                       
1935.0           NaN          NaN          NaN          NaN          NaN   
1936.0           NaN          NaN          NaN          NaN          NaN   
1940.0           NaN          NaN          NaN          NaN          NaN   
1948.0           NaN          NaN          NaN          NaN          NaN   
1953.0           NaN          NaN          NaN          NaN          NaN   
1955.0           NaN          NaN          NaN          NaN          NaN   
1961.0           NaN          NaN          NaN          NaN          NaN   
1962.0           NaN          NaN          NaN          NaN          NaN   
1963.0           NaN          NaN          NaN          NaN          NaN   
1964.0           NaN          NaN          NaN          NaN          NaN   

station  USC00014502  USC00016129  USC00017025  USC00091425  USC00092159  \
year                                                                       
1935.0           NaN          NaN          NaN          NaN          NaN   
1936.0           NaN          NaN          NaN          NaN          NaN   
1940.0           NaN          NaN    41.699198          NaN          NaN   
1948.0           NaN          NaN     3.486062          NaN          NaN   
1953.0           NaN          NaN          NaN          NaN          NaN   
1955.0           NaN          NaN          NaN          NaN          NaN   
1961.0           NaN          NaN          NaN          NaN          NaN   
1962.0           NaN          NaN    17.057494          NaN          NaN   
1963.0           NaN          NaN          NaN          NaN          NaN   
1964.0           NaN          NaN    12.917347          NaN          NaN   

station     ...       USC00097201  USC00098535  USC00098657  USC00098661  \
year        ...                                                            
1935.0      ...               NaN          NaN          NaN          NaN   
1936.0      ...               NaN          NaN          NaN          NaN   
1940.0      ...               NaN          NaN          NaN          NaN   
1948.0      ...               NaN          NaN          NaN          NaN   
1953.0      ...               NaN          NaN          NaN          NaN   
1955.0      ...               NaN          NaN          NaN          NaN   
1961.0      ...               NaN          NaN          NaN          NaN   
1962.0      ...               NaN     3.573302          NaN          NaN   
1963.0      ...               NaN          NaN          NaN          NaN   
1964.0      ...               NaN          NaN          NaN          NaN   

station  USC00099124  USC00099291  USC00099506  USW00003813  USW00013860  \
year                                                                       
1935.0           NaN          NaN          NaN          NaN          NaN   
1936.0           NaN     8.429300          NaN          NaN          NaN   
1940.0           NaN    13.335233          NaN          NaN          NaN   
1948.0           NaN          NaN          NaN          NaN          NaN   
1953.0           NaN          NaN          NaN    -0.891371          NaN   
1955.0           NaN          NaN          NaN          NaN          NaN   
1961.0           NaN          NaN          NaN          NaN          NaN   
1962.0           NaN          NaN          NaN          NaN          NaN   
1963.0           NaN          NaN          NaN          NaN          NaN   
1964.0           NaN          NaN          NaN          NaN          NaN   

station  USW00093842  
year                  
1935.0           NaN  
1936.0           NaN  
1940.0           NaN  
1948.0           NaN  
1953.0           NaN  
1955.0           NaN  
1961.0           NaN  
1962.0           NaN  
1963.0           NaN  
1964.0           NaN  

[10 rows x 38 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [44]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  98.6660902199
RMS removing mean-by-station=  66.6624088622
RMS removing mean-by-year   =  8.96181070928


In [45]:
def MS(Mat):
    return np.nanmean(Mat**2)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station),' fraction explained=',1-MS(tbl_minus_station)/MS(year_station_table)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year),' fraction explained=',1-MS(tbl_minus_year)/MS(year_station_table)


total MS                   =  9734.99735929
MS removing mean-by-station=  4443.87675531  fraction explained= 0.543515360991
MS removing mean-by-year   =  80.3140511889  fraction explained= 0.991749966823


In [46]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 98.6660902199
0 after removing mean by year    = 8.96181070928
0 after removing mean by stations= 6.89769065334
1 after removing mean by year    = 6.37808118594
1 after removing mean by stations= 6.11179440811
2 after removing mean by year    = 5.95291801339
2 after removing mean by stations= 5.84826245804
3 after removing mean by year    = 5.7740272747
3 after removing mean by stations= 5.71831508189
4 after removing mean by year    = 5.67471021168
4 after removing mean by stations= 5.63952182467
